In [1]:
import numpy as np

class SimpleDecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

    def _gini(self, y):
        """Tính độ tinh khiết Gini: càng nhỏ càng tốt"""
        if len(y) == 0:
            return 0
        probs = np.bincount(y) / len(y)
        return 1 - np.sum(probs ** 2)

    def _best_split(self, X, y):
        """Tìm đặc trưng & ngưỡng chia tốt nhất"""
        best_gini = float('inf')
        best_split = {}
        n_features = X.shape[1]

        for feat_idx in range(n_features):
            thresholds = np.unique(X[:, feat_idx])
            for th in thresholds:
                # Chia thành 2 nhóm
                left_mask = X[:, feat_idx] <= th
                right_mask = ~left_mask

                if np.sum(left_mask) == 0 or np.sum(right_mask) == 0:
                    continue

                # Tính Gini tổng sau chia
                gini_left = self._gini(y[left_mask])
                gini_right = self._gini(y[right_mask])
                n_left, n_right = np.sum(left_mask), np.sum(right_mask)
                weighted_gini = (n_left * gini_left + n_right * gini_right) / len(y)

                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_split = {
                        'feature_idx': feat_idx,
                        'threshold': th,
                        'gini': best_gini
                    }
        return best_split

    def _build_tree(self, X, y, depth=0):
        """Xây dựng cây đệ quy"""
        n_samples, n_classes = len(y), len(np.unique(y))

        # Điều kiện dừng
        if (depth >= self.max_depth or 
            n_samples < self.min_samples_split or 
            n_classes == 1):
            leaf_value = np.bincount(y).argmax()
            return {'leaf': True, 'value': leaf_value}

        # Tìm cách chia tốt nhất
        best_split = self._best_split(X, y)
        if not best_split:  # không chia được
            leaf_value = np.bincount(y).argmax()
            return {'leaf': True, 'value': leaf_value}

        # Chia dữ liệu
        left_mask = X[:, best_split['feature_idx']] <= best_split['threshold']
        right_mask = ~left_mask

        # Đệ quy xây con trái/phải
        left_child = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_child = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        return {
            'leaf': False,
            'feature_idx': best_split['feature_idx'],
            'threshold': best_split['threshold'],
            'left': left_child,
            'right': right_child
        }

    def fit(self, X, y):
        self.tree = self._build_tree(np.array(X), np.array(y))

    def _predict_sample(self, x, tree):
        if tree['leaf']:
            return tree['value']
        if x[tree['feature_idx']] <= tree['threshold']:
            return self._predict_sample(x, tree['left'])
        else:
            return self._predict_sample(x, tree['right'])

    def predict(self, X):
        return np.array([self._predict_sample(x, self.tree) for x in X])

# --- Thử nghiệm ---
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Huấn luyện mô hình tự code
dt_model = SimpleDecisionTree(max_depth=3, min_samples_split=2)
dt_model.fit(X_train, y_train)
y_pred = dt_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"✅ Độ chính xác (Decision Tree tự code): {acc:.2%}")

# So sánh với sklearn
from sklearn.tree import DecisionTreeClassifier
sk_dt = DecisionTreeClassifier(max_depth=3, random_state=42)
sk_dt.fit(X_train, y_train)
sk_acc = sk_dt.score(X_test, y_test)
print(f"🔍 Sklearn DecisionTree độ chính xác: {sk_acc:.2%}")

# (Tùy chọn) Vẽ cây từ sklearn để xem trực quan
# from sklearn.tree import plot_tree
# import matplotlib.pyplot as plt
# plt.figure(figsize=(12,8))
# plot_tree(sk_dt, feature_names=iris.feature_names, class_names=iris.target_names, filled=True)
# plt.show()

✅ Độ chính xác (Decision Tree tự code): 95.56%
🔍 Sklearn DecisionTree độ chính xác: 100.00%


🔹 1. Nguyên lý hoạt động (không công thức nặng!)
🎯 Mục tiêu:
Chia dữ liệu thành các nhóm thuần nhất (cùng nhãn) bằng cách đặt câu hỏi có/không liên tục về các đặc trưng.

🧠 Ý tưởng chính — như chơi trò "20 câu hỏi":
"Chiều dài cánh hoa > 2.5 cm không?"
→ Nếu có, đi nhánh phải; nếu không, đi nhánh trái.
→ Lặp lại cho đến khi đủ chắc chắn để gán nhãn. 

🔍 Cấu trúc cây:
Nút gốc (root): Câu hỏi đầu tiên (quan trọng nhất).
Nút trong (internal node): Câu hỏi tiếp theo.
Lá (leaf): Nhãn dự đoán (ví dụ: "setosa", "spam", "bị bệnh").
💡 Làm sao chọn "câu hỏi tốt nhất"?
Cây sẽ chọn đặc trưng và ngưỡng sao cho sau khi chia, các nhóm con càng "thuần" càng tốt.
Dùng độ tinh khiết (purity) để đo:
Gini impurity (phổ biến, đơn giản)
Information gain (dựa trên entropy – từ lý thuyết thông tin)
🌟 Không cần nhớ công thức!
Chỉ cần hiểu: "Chia sao cho mỗi nhánh càng ít lẫn lộn nhãn càng tốt." 

✅ Ưu điểm:
Cực kỳ dễ hiểu và trực quan → có thể vẽ ra như sơ đồ.
Không cần chuẩn hóa dữ liệu.
Xử lý được cả đặc trưng số và phân loại.
Tự động chọn đặc trưng quan trọng.
❌ Hạn chế:
Dễ overfit (học thuộc nhiễu trong dữ liệu) → cây quá sâu, quá phức tạp.
Không ổn định: thay đổi nhỏ trong dữ liệu → cây khác hẳn.
Hiệu suất dự đoán kém hơn so với ensemble (Random Forest, XGBoost).

🔹 3. Khi nào dùng Decision Tree?
Bạn cần
giải thích mô hình
cho người không rành ML (ví dụ: bác sĩ, quản lý)
Dữ liệu
rất nhiễu
→ cây dễ overfit
Dữ liệu
hỗn hợp
(số + phân loại)
Cần
độ chính xác cao nhất
→ hãy dùng
Random Forest / XGBoost
Làm
baseline nhanh
, không cần chuẩn hóa
Bài toán
rủi ro cao
(y tế, tài chính) → cây đơn lẻ
không ổn định
Muốn
hiểu đặc trưng nào quan trọng
Dữ liệu
rất lớn
→ cây đơn có thể chậm và kém hiệu quả